In [3]:
from Layers import FullyConnected, Helpers, ReLU, SoftMax, Loss
from Optimization import Optimizers
import copy
import matplotlib.pyplot as plt
import numpy as np

In [65]:
class NeuralNetwork:
    def __init__(self, optimizer):
        self.optimizer = optimizer
        self.loss = []
        self.layers = []
        self.data_layer = None
        self.loss_layer = None
        
    def forward(self):
        input_tensor, self.label_tensor = self.data_layer.next()
        output = input_tensor
        for layer in self.layers:
            output = layer.forward(output)
        return output
    
    def backward(self, label_tensor):
        error = self.loss_layer.backward(label_tensor)
        for layer in reversed(self.layers):
            error = layer.backward(error)
        
    def append_layer(self, layer):
        if layer.trainable:
            layer.optimizer = copy.deepcopy(self.optimizer)
        self.layers.append(layer)
        
    def train(self, iterations):
        for _ in range(iterations):
            output = self.forward()
            self.loss.append(self.loss_layer.forward(output, self.label_tensor))
            self.backward(self.label_tensor)
            
    def test(self, input_tensor):
        output = input_tensor
        for layer in self.layers:
            output = layer.forward(output)
        return output

In [67]:
import warnings
with warnings.catch_warnings():
 # ignore all caught warnings
 warnings.filterwarnings("ignore")

In [69]:
net = NeuralNetwork(Optimizers.Sgd(1))
categories = 3
input_size = 4
net.data_layer = Helpers.IrisData(50)
net.loss_layer = Loss.CrossEntropyLoss()
fcl_1 = FullyConnected.FullyConnected(input_size, categories)
net.append_layer(fcl_1)
net.append_layer(ReLU.ReLU())
fcl_2 = FullyConnected.FullyConnected(categories, categories)
net.append_layer(fcl_2)
net.append_layer(SoftMax.SoftMax())

out = net.forward()
out2 = net.forward()

c:\Users\ADE17\anaconda3\anaconda2\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
